In [1]:
from glob import glob
dataset_dir = 'UTKFace/'
images = glob(dataset_dir+'*.jp*g')
print(len(images))

23708


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [4]:
def decode_img_in_binary(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [5]:
def get_label(file_path):
    Y = np.zeros((117))
    name = file_path[8:]
    x = name.split('_')
    Y[int(x[0].strip())-1] = 1
    return Y

In [6]:
def process_path(file_path,label):
    img = tf.io.read_file(file_path)
    img = decode_img_in_binary(img)
#     Y= get_label(file_path)
    return img, label

In [7]:
labels = []
for img_name in images:
    labels.append(get_label(img_name))

In [9]:
print(len(images), len(labels))

23708 23708


In [10]:
train_images = images[:(2*len(images)//3)+1]
test_images = images[(2*len(images)//3)+1:]
train_labels = labels[:(2*len(images)//3)+1]
test_labels = labels[(2*len(images)//3)+1:]


In [11]:
print(len(train_images))
print(len(test_images))

15806
7902


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
train_dataset = train_dataset.shuffle(len(train_images))
train_dataset =train_dataset.map(process_path, num_parallel_calls=4)
train_dataset = train_dataset.batch(50)

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_labels))
test_dataset = test_dataset.shuffle(len(test_images))
test_dataset =test_dataset.map(process_path, num_parallel_calls=4)
test_dataset = test_dataset.batch(50)

***Model Initialization***

In [14]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [15]:
model = VGG16(input_shape=[224,224,3],weights='imagenet', include_top=False)
for layer in model.layers:
    layer.trainable=False

In [16]:
x=Flatten()(model.output)
x=Dense(512,activation='relu')(x)
x=Dense(128,activation='relu')(x)
prediction=Dense(117,activation='softmax')(x)
final_model=Model(inputs=model.input,outputs=prediction)
final_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
import keras
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
final_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
for image,label in train_dataset:
    final_model.fit(image.numpy(),label.numpy(),epochs=2)

Epoch 1/2
50/50 [==============================] - 10s 205ms/step - loss: 4.9270 - accuracy: 0.0600
Epoch 2/2
50/50 [==============================] - 10s 208ms/step - loss: 4.0956 - accuracy: 0.1000
Epoch 1/2
50/50 [==============================] - 10s 209ms/step - loss: 4.5217 - accuracy: 0.1000
Epoch 2/2
50/50 [==============================] - 10s 210ms/step - loss: 3.7372 - accuracy: 0.1200
Epoch 1/2
50/50 [==============================] - 11s 211ms/step - loss: 4.4920 - accuracy: 0.1400
Epoch 2/2
50/50 [==============================] - 10s 207ms/step - loss: 3.7716 - accuracy: 0.1800
Epoch 1/2
50/50 [==============================] - 10s 209ms/step - loss: 4.5368 - accuracy: 0.1200
Epoch 2/2
50/50 [==============================] - 10s 207ms/step - loss: 3.9342 - accuracy: 0.1600
Epoch 1/2
50/50 [==============================] - 10s 209ms/step - loss: 4.3078 - accuracy: 0.0800
Epoch 2/2
50/50 [==============================] - 11s 221ms/step - loss: 3.6224 - accuracy: 0.1800


Epoch 1/2
50/50 [==============================] - 10s 207ms/step - loss: 4.1597 - accuracy: 0.0400
Epoch 2/2
50/50 [==============================] - 11s 222ms/step - loss: 3.7048 - accuracy: 0.1400
Epoch 1/2
50/50 [==============================] - 11s 214ms/step - loss: 3.8291 - accuracy: 0.1200
Epoch 2/2
50/50 [==============================] - 11s 221ms/step - loss: 3.3545 - accuracy: 0.2200
Epoch 1/2
50/50 [==============================] - 11s 218ms/step - loss: 4.4711 - accuracy: 0.1000
Epoch 2/2
50/50 [==============================] - 11s 215ms/step - loss: 3.8381 - accuracy: 0.1400
Epoch 1/2
50/50 [==============================] - 11s 223ms/step - loss: 4.1351 - accuracy: 0.1600
Epoch 2/2


In [14]:
for img, lab in dataset:
    final_model.evaluate(img.numpy(),lab.numpy())

100/100 [==============================] - 21s 205ms/step
